```{r setup, include=FALSE}
knitr::opts_chunk$set(echo = FALSE)
library(this.path)
library(r4ss)
library(ss3diags)
library(tidyverse)
library(gt)


```

**This is a summary report for the `r params$species` `r params$scenario` model run.** 

```{r}
#file.path(params$report, params$species, params$file_dir)
report. <- SS_output(dir = getwd(), verbose = FALSE, printstats = FALSE)

```

# Model Output 

::: {.panel-tabset}

## Input Data
```{r }
SSplotData(report.)
```

## Convergence Check
```{r}
data.frame(Converged = file.exists(file.path(getwd(), "ss.std")), 
           MaxGrad = report.$maximum_gradient_component)

report.$warnings
```


## Fit to Model 
### CPUE

```{r}

n.indices <- length(unique(report.$cpue$Fleet))
n.comps <- length(unique(report.$lendbase$Fleet))

report.$index_variance_tuning_check %>% 
  select(Fleet, fleetname, N, RMSE, VarAdj, New_VarAdj) %>% 
  gt()

for(i in 1:n.indices){
  sspar(mfrow = c(1,2))
  SSplotIndices(report., subplots = 2, fleets = i)
  SSplotRunstest(report., subplots = "cpue", add = TRUE, indexselect = i)
}

for(i in 1:n.comps){
  sspar(mfrow = c(1,1))
  SSplotJABBAres(report., subplots = "cpue", add = TRUE, indexselect = i)
}

```

### Length Comp
```{r}
SS_tune_comps(report., option = "Francis", dir = getwd(), write =F, verbose =F) %>% 
  select('#Factor', Fleet, New_Var_adj, Type, Name) %>% 
  gt()

sspar(mfrow = c(1,2))
SSplotRunstest(report., subplots = "len", add = TRUE, indexselect = 1)
SSplotJABBAres(report., subplots = "len", add = TRUE, indexselect = 1)

SSplotComps(report., subplots = 8, fleets = 1)
SSplotComps(report., subplots = 21, fleets = 1)
SSplotComps(report., subplots = 1, fleets = 1)


```

## Retrospective and Hindcasting  
### Retrospective 
```{r message=FALSE, eval = TRUE, error=TRUE}
if(dir.exists(file.path(getwd(), "Retrospectives"))){
  
  retro.dirs <- list.files(file.path(getwd(), "Retrospectives"),
                           full.names = TRUE)
  mod.retro <- SSgetoutput(dirvec = retro.dirs, verbose = FALSE)


retrosum <- SSsummarize(mod.retro, verbose = FALSE)
retrocomp <- SSretroComps(mod.retro)

sspar(mfrow = c(1,2))
SSplotRetro(retrosum,
            subplots = "SSB",
            add = TRUE)

SSplotRetro(retrosum,
            subplots = "F",
            add =TRUE)
}else{
  print("No retrospective runs were found")
}

```

### Hindcasting
```{r eval =TRUE,error=TRUE}
if(dir.exists(file.path(getwd(), "Retrospectives"))){
  for(i in 1:n.indices){
    sspar(mfrow = c(1,2))
    SSplotHCxval(retrosum, subplots = "cpue", add = TRUE, indexselect = i)
  }
  
  for(i in 1:n.comps){
    sspar(mfrow = c(1,2))
    SSplotHCxval(retrocomp, subplots = "len", add = TRUE, indexselect = i)
  }
  
}else{
  print("No information for hindcast was found")
  
}

```


## Recruitment Deviations 
```{r}
SSplotRecdevs(report., subplots = c(2,3))
```


## Likelihood Profile

```{r eval=TRUE}
if(dir.exists(file.path(getwd(), paste0(params$profile, "_profile")))){
  
r0.vec <- params$profile_vec
Nprofile <- length(r0.vec)
profile.str <- substr(params$profile, start=1, stop=5)
print(profile.str)
# read the output files (with names like Report1.sso, Report2.sso, etc.)
profile.dirs <- file.path(getwd(), paste0(params$profile, "_profile"))
profilemodels <- SSgetoutput(dirvec = profile.dirs,
                             keyvec = 1:Nprofile, verbose = FALSE)
# summarize output
profilesummary <- SSsummarize(profilemodels, verbose = FALSE)

# OPTIONAL COMMANDS TO ADD MODEL WITH PROFILE PARAMETER ESTIMATED
MLEmodel <- SS_output(file.path(getwd()),
                      verbose = FALSE, printstats = FALSE)
profilemodels[["MLE"]] <- MLEmodel
profilesummary <- SSsummarize(profilemodels, verbose = FALSE)
# END OPTIONAL COMMANDS

# plot profile using summary created above
SSplotProfile(profilesummary, # summary object
  profile.string = profile.str, # substring of profile parameter
  profile.label = params$profile
)

PinerPlot(profilesummary, component = "Surv_like",
          profile.string = profile.str,
          profile.label = params$profile,
          main = "Changes in survey likelihood by fleet")

}else{
  print("No likelihood runs were found")
  
}



```

## Management Quantities  
```{r eval=TRUE}

mvln <- SSdeltaMVLN(report.)
SSplotEnsemble(mvln$kb, subplots = c("stock", "harvest", "SSB", "F"), add = TRUE)
dev.off()

```


## Jitter  
```{r message = FALSE, warning=FALSE}
if(dir.exists(file.path(getwd(), "jitter"))){
  
  jittermods <- SSgetoutput(keyvec=1:params$Njitter,
                            dirvec=file.path(getwd(), "jitter"),
                            getcomp=FALSE, getcovar=F, verbose =FALSE)
  jittermods[["base"]] <- SS_output(file.path(getwd()),
                                    verbose = FALSE, printstats = FALSE)
  jittersum <- SSsummarize(jittermods, verbose = FALSE)

  baselike <- jittersum$likelihoods %>%
    select(base, Label) %>%
    filter(str_detect(Label, "TOTAL")) %>%
    pull(base)

  
 likes <- jittersum$likelihoods %>%
    pivot_longer(cols = -Label, names_to = "Run", values_to = "Likelihood") %>%
    filter(str_detect(Label, "TOTAL")) %>%
    mutate(Run = seq(1, params$Njitter + 1)) %>%
    ggplot(aes(x = Run, y = Likelihood)) +
    geom_point() +
    geom_hline(yintercept = baselike)

  likes
  
}else{
  print("No jitter runs were found.")
}
  
```


    
```{r  message = FALSE, warning=FALSE}  
if(dir.exists(file.path(getwd(), "jitter"))){
   SSplotComparisons(jittersum, subplots = c(2,8,10), new = F)
}
```

:::